# PRIMO 2022 ML Homework 4<br>
### Estimating the issue priority in the bug tracker

Задача: предсказать, имеет ли проблема высокий приоритет, учитывая ее описание и метаданные.<br>
Данные: взяты с сайта youtrack.jetbrains.com.<br>
Оценка качества: F1 мера

Метрика оценки качества для соревнования: F1 мера для класса 1.<br>
<br>
**Формат файла с предсказаниями<br>**
Для каждой задачи в наборе test итоговый файл с предсказаниями должен содержать два значения: id (id задачи, взятый из столбца `id` в наборе данных test) и предсказание приоритета задачи (1, если приоритет высокий, или 0, если приоритет низкий).<br>
<br>
Файл должен иметь заголовок и следующий формат:<br>
<br>
id,is_high_priority<br>
0,1<br>
1,1<br>
2,1<br>

In [132]:
import pandas as pd
import numpy as np

import warnings
import json
import operator

from tqdm import tqdm

from sklearn.model_selection import train_test_split


from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
#np.set_printoptions(precision=3)
#pd.set_option('precision', 3)



In [133]:
dataFrameMapper_data = pd.read_csv('test.csv', index_col='id')

In [134]:
train_data = pd.read_csv('train.csv', index_col='id')



In [135]:
train_data1 = train_data.sample(frac = 1)
chunks = np.array_split(train_data1, 10)
train_data1 = chunks[0]

In [136]:
train_data1.summary.fillna('', inplace=True)
train_data1.description.fillna('', inplace=True)


In [137]:
dataFrameMapper_data.summary.fillna('', inplace=True)
dataFrameMapper_data.description.fillna('', inplace=True)

In [138]:
json_fields = pd.json_normalize(train_data1.customFields.map(json.loads).map(lambda x: {field['name']: field['value'] for field in x}))
json_fields.set_index(train_data1.index, inplace=True)
json_fields.fillna('',inplace=True)



In [139]:
json_fields['Subsystem.name'].value_counts()

                               2233
User Interface                  717
Editor. Editing Text            435
Build. Maven                    355
Lang. Flash and Flex            318
                               ... 
Java. Debugger. Memory View       1
Clouds. CloudBees                 1
Java. Type Migration              1
Groovy. Inspections               1
Lang. JSON                        1
Name: Subsystem.name, Length: 160, dtype: int64

In [140]:
json_columns = ['Type.name','State.name','Subsystem.name','Assignee.login']
train_data1 = train_data1.join(json_fields[json_columns], how='outer', lsuffix='_left')


train_data1['reporter_name'] = train_data1.reporter.map(json.loads).map(operator.itemgetter('login'))
train_data1['Year'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').year)
train_data1['Month'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').month)
train_data1['Day'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').day)
train_data1['Hour'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').hour)
prep_train_data = train_data1.drop(labels=['reporter', 'created', 'customFields', 'links'], axis=1)

In [141]:
prep_train_data

,summary,description,is_high_priority,Type.name,State.name,Subsystem.name,Assignee.login,reporter_name,Year,Month,Day,Hour
id,,,,,,,,,,,,
25-534341,Subversion commit with native client: do not t...,1 set Subversion config directory to some non-...,False,Bug,Fixed,Version Control. Subversion,irengrig,irengrig,2013,4,2,11
25-303513,Kill Selected Region no longer deletes current...,"Previously using the ""Kill Selected Region"" ac...",False,Bug,Declined,Editor. Editing Text,denis.zhdanov,snstanton,2011,4,20,22
25-61040,Intellij suddenly hanged up(stopped responding),Intellij suddenly hanged up(stopped responding...,False,Bug,Obsolete,,Sascha_Weinreuter,igorewka,2009,2,24,6
25-60793,"Ability to have always turned on ""Highligh Usa...",When my cursors is on the word. Some kind of o...,False,Feature,Declined,Java. Intention Actions,,leszekgruchala,2009,1,22,17
25-1176131,Out of Memory when importing really large Grad...,I've encountered a Out Of Memory situation whe...,False,Bug,Obsolete,Build. Gradle,vsoroka,jwthewes,2016,4,7,14
...,...,...,...,...,...,...,...,...,...,...,...,...
25-1991627,Run causes to loose focus from editor,"1) Click on the editor\n2) Launch ""Run"" from h...",False,Bug,Submitted,User Interface. Focus,fokin,cyrsap,2018,5,18,9
25-62181,Issue fixed in IDEADEV-34862 has been regressed,"The issue in ""IDEADEV-34862 Mac OS X: Mnemonic...",False,Bug,Duplicate,User Interface,,danieljimenez,2009,8,3,23
25-76242,IDEA should not ping me with running bg tasks ...,Please see attached screenshot,False,Bug,Submitted,,cdr,alexey.pegov,2009,10,22,9


In [142]:
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import re

In [143]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

*сначала происходит лемматизация слов, и потом они отфильтровываются по условиям, так работать не должно*

In [144]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
       return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if re.fullmatch(r'[A-Za-z]{4,}', t) and t not in STOP_WORDS]

In [145]:
X_train_text = prep_train_data.summary.astype(str) + " " + prep_train_data.description.astype(str)


In [146]:
X_train_text

id
25-534341     Subversion commit with native client: do not t...
25-303513     Kill Selected Region no longer deletes current...
25-61040      Intellij suddenly hanged up(stopped responding...
25-60793      Ability to have always turned on "Highligh Usa...
25-1176131    Out of Memory when importing really large Grad...
                                    ...                        
25-1991627    Run causes to loose focus from editor 1) Click...
25-62181      Issue fixed in IDEADEV-34862 has been regresse...
25-76242      IDEA should not ping me with running bg tasks ...
25-1461693    Usual class to reflection What would be really...
25-62165      ActionScript Editor does not support temporal ...
Length: 10537, dtype: object

In [147]:
tfidf_vec_summary = TfidfVectorizer(min_df=.01, max_df=0.75,
                                    tokenizer=LemmaTokenizer(),
                             )
tfidf_vec_summary.fit(X_train_text)
train_text_prep = pd.DataFrame(tfidf_vec_summary.transform(X_train_text).todense(), index=X_train_text.index,
                               columns=tfidf_vec_summary.get_feature_names_out())
train_text_prep



c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,ability,able,access,action,active,actual,actually,added,adding,additional,...,warning,webstorm,window,word,work,workaround,worked,working,write,wrong
id,,,,,,,,,,,,,,,,,,,,,
25-534341,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25-303513,0.000000,0.0,0.0,0.182387,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25-61040,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25-60793,0.287674,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.287323,0.0,0.0,0.0,0.0,0.0,0.0
25-1176131,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1991627,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25-62181,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25-76242,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
prep_train_data = prep_train_data.join(train_text_prep)

In [149]:
prep_train_data = prep_train_data.drop(['summary','description'], axis=1)

In [150]:
prep_train_data

,is_high_priority,Type.name,State.name,Subsystem.name,Assignee.login,reporter_name,Year,Month,Day,Hour,...,warning,webstorm,window,word,work,workaround,worked,working,write,wrong
id,,,,,,,,,,,,,,,,,,,,,
25-534341,False,Bug,Fixed,Version Control. Subversion,irengrig,irengrig,2013,4,2,11,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25-303513,False,Bug,Declined,Editor. Editing Text,denis.zhdanov,snstanton,2011,4,20,22,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25-61040,False,Bug,Obsolete,,Sascha_Weinreuter,igorewka,2009,2,24,6,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25-60793,False,Feature,Declined,Java. Intention Actions,,leszekgruchala,2009,1,22,17,...,0.0,0.0,0.0,0.287323,0.0,0.0,0.0,0.0,0.0,0.0
25-1176131,False,Bug,Obsolete,Build. Gradle,vsoroka,jwthewes,2016,4,7,14,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1991627,False,Bug,Submitted,User Interface. Focus,fokin,cyrsap,2018,5,18,9,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25-62181,False,Bug,Duplicate,User Interface,,danieljimenez,2009,8,3,23,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25-76242,False,Bug,Submitted,,cdr,alexey.pegov,2009,10,22,9,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [151]:
#pd.options.display.max_rows = 200



tfidf_vec_desc = TfidfVectorizer(min_df=.01, max_df=0.5,
                                   # token_pattern=r'[A-Za-z]{2,}',
                                    tokenizer=LemmaTokenizer(),
                                    stop_words=STOP_WORDS)
tfidf_vec_desc.fit(X_train_description)
X_train_description_sample = pd.DataFrame(tfidf_vec_desc.transform(X_train_description).todense(),
                                          columns=tfidf_vec_desc.get_feature_names_out())
X_train_description_sample

In [152]:
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

\# todo сократить размерность. Лемматизация, регулярные выражения поумнее, пороги для векторизации, объединить слова описания и заголовка?
отбор признаков по модели

In [153]:
from sklearn_pandas import DataFrameMapper

preprocessor = DataFrameMapper([
                                (['Type.name'], OneHotEncoder(sparse=False, handle_unknown='ignore')),
                                (['reporter_name'], OneHotEncoder(sparse=False, handle_unknown='ignore')),
                                (['Subsystem.name'], OneHotEncoder(sparse=False, handle_unknown='ignore')),
                                ('Year', None),
                                ('Month', None),
                                ('Day', None),
                                ('Hour', None),
                                ],
                               input_df=True,
                               df_out=True)



In [154]:
X, Y = prep_train_data.drop(['is_high_priority'], axis=1), prep_train_data['is_high_priority']


In [155]:
dataFrameMapper_data = preprocessor.fit_transform(X)


c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [156]:
dataFrameMapper_data



,Type.name_x0_Auto-reported Exception,Type.name_x0_Bug,Type.name_x0_Configuration Problem,Type.name_x0_Cosmetics,Type.name_x0_Exception,Type.name_x0_Feature,Type.name_x0_Meta Issue,Type.name_x0_Performance Problem,Type.name_x0_Security Problem,Type.name_x0_Support Request,...,Subsystem.name_x0_Version Control. Mercurial,Subsystem.name_x0_Version Control. Perforce,Subsystem.name_x0_Version Control. StarTeam (archived),Subsystem.name_x0_Version Control. Subversion,Subsystem.name_x0_Version Control. TFS (archived),Subsystem.name_x0_Version Control. VSS (archived),Year,Month,Day,Hour
id,,,,,,,,,,,,,,,,,,,,,
25-534341,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2013,4,2,11
25-303513,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2011,4,20,22
25-61040,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2009,2,24,6
25-60793,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2009,1,22,17
25-1176131,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2016,4,7,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1991627,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2018,5,18,9
25-62181,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2009,8,3,23
25-76242,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2009,10,22,9


In [157]:
#dataFrameMapper_data.to_csv('dataMapperData.csv')

In [158]:
Y = Y.astype(np.int8)

In [159]:
X_train, X_test, y_train, y_test = train_test_split(dataFrameMapper_data, Y,
                                                    stratify=Y,
                                                    random_state=3,
                                                    test_size=0.25,
                                                    shuffle=True)

In [160]:
X_train

,Type.name_x0_Auto-reported Exception,Type.name_x0_Bug,Type.name_x0_Configuration Problem,Type.name_x0_Cosmetics,Type.name_x0_Exception,Type.name_x0_Feature,Type.name_x0_Meta Issue,Type.name_x0_Performance Problem,Type.name_x0_Security Problem,Type.name_x0_Support Request,...,Subsystem.name_x0_Version Control. Mercurial,Subsystem.name_x0_Version Control. Perforce,Subsystem.name_x0_Version Control. StarTeam (archived),Subsystem.name_x0_Version Control. Subversion,Subsystem.name_x0_Version Control. TFS (archived),Subsystem.name_x0_Version Control. VSS (archived),Year,Month,Day,Hour
id,,,,,,,,,,,,,,,,,,,,,
25-707155,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2014,6,14,12
25-2410587,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2019,7,5,9
25-478182,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2012,10,18,12
25-224648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2010,11,28,22
25-309134,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2011,5,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-2393932,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2019,6,19,21
25-577719,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2013,7,3,0
25-494163,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2012,12,8,22


In [161]:
# https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB, MultinomialNB, ComplementNB

In [198]:
from sklearn.preprocessing import StandardScaler

std_scaler = MinMaxScaler()

X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

X_train_scaled

array([[0.        , 1.        , 0.        , ..., 0.45454545, 0.43333333,
        0.52173913],
       [0.        , 1.        , 0.        , ..., 0.54545455, 0.13333333,
        0.39130435],
       [0.        , 1.        , 0.        , ..., 0.81818182, 0.56666667,
        0.52173913],
       ...,
       [0.        , 1.        , 0.        , ..., 1.        , 0.23333333,
        0.95652174],
       [0.        , 1.        , 0.        , ..., 0.72727273, 0.96666667,
        0.47826087],
       [0.        , 0.        , 0.        , ..., 0.27272727, 0.63333333,
        0.47826087]])

# todo надо бужет скэйлить!
from sklearn.ensemble import RandomForestRegressor
clf = SGDClassifier()

clf.fit(X_train_scaled,y_train)

In [227]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [228]:
SVC_model = SVC()
# В KNN-модели нужно указать параметр n_neighbors
# Это число точек, на которое будет смотреть
# классификатор, чтобы определить, к какому классу принадлежит новая точка
KNN_model = KNeighborsClassifier(n_neighbors=2)


SVC_model.fit(X_train, y_train)
KNN_model.fit(X_train, y_train)


SVC_prediction = SVC_model.predict(X_test)
KNN_prediction = KNN_model.predict(X_test)

In [231]:
logReg = KNeighborsClassifier(n_neighbors=2)
logReg.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=2)

In [224]:
predictions = logReg.predict(X_test_scaled)

In [218]:
proba = logReg.predict_proba(X_test_scaled)[:, 1]

In [219]:
threshold = 0.03
f1_score(y_true=y_test, y_pred=proba > threshold)

0.03278688524590164

In [230]:
f1_score(y_true=y_test, y_pred=KNN_prediction)

0.0

In [232]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05, .001,.0023, 0.003])

resultsProba = []
for params in tqdm(ParameterGrid(param_grid)):

  #  prediction = logReg.predict_proba(X_test_scaled)[:,1]
    resultsProba.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=logReg.predict_proba(X_train_scaled)[:,1] > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=logReg.predict_proba(X_test_scaled)[:,1] > params['threshold']),

    ))


100%|██████████| 11/11 [00:58<00:00,  5.35s/it]


In [ ]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05, .001,.0023, 0.003])

resultsProba = []
for params in tqdm(ParameterGrid(param_grid)):

  #  prediction = logReg.predict_proba(X_test_scaled)[:,1]
    resultsProba.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=logReg.predict_proba(X_train_scaled)[:,1] > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=logReg.predict_proba(X_test_scaled)[:,1] > params['threshold']),

    ))


In [223]:
pd.DataFrame(resultsProba).sort_values('test_f1').style.bar(vmin=0, vmax=1)

,parameters,train_f1,test_f1
0,{'threshold': 0.1},0.000000,0.000000
1,{'threshold': 0.2},0.000000,0.000000
2,{'threshold': 0.3},0.000000,0.000000
3,{'threshold': 0.4},0.000000,0.000000
4,{'threshold': 0.5},0.000000,0.000000
7,{'threshold': 0.05},0.016901,0.000000
10,{'threshold': 0.03},0.060606,0.032787
5,{'threshold': 0.01},0.245283,0.051282
6,{'threshold': 0.005},0.381119,0.099617
9,{'threshold': 0.0023},0.324222,0.114286




prediction = clf.predict_proba(X_test)[:,1]
#print (f'Test f1 = {f1_score(y_true=y_test, y_pred=nb_pipeline.predict(X_test))}')

prediction

score = clf.score(X_test, y_test)
score

In [68]:
X_test

,Type.name_x0_Auto-reported Exception,Type.name_x0_Bug,Type.name_x0_Configuration Problem,Type.name_x0_Cosmetics,Type.name_x0_Exception,Type.name_x0_Feature,Type.name_x0_Meta Issue,Type.name_x0_Performance Problem,Type.name_x0_Security Problem,Type.name_x0_Support Request,...,Subsystem.name_x0_Version Control. Log,Subsystem.name_x0_Version Control. Mercurial,Subsystem.name_x0_Version Control. Perforce,Subsystem.name_x0_Version Control. StarTeam (archived),Subsystem.name_x0_Version Control. Subversion,Subsystem.name_x0_Version Control. TFS (archived),Year,Month,Day,Hour
id,,,,,,,,,,,,,,,,,,,,,
25-678404,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2014,3,18,11
25-1269972,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2016,7,29,13
25-558464,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2013,5,23,20
25-643012,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2013,12,5,16
25-496273,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2012,12,17,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1505820,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2017,4,6,14
25-1476543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2017,3,14,9
25-2254722,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2019,2,19,16


In [79]:

minMaxTest_Y = prep_train_data.is_high_priority

In [84]:
test_minMaxdf_X = prep_train_data.summary.astype(str) + " " + prep_train_data.description.astype(str)

In [85]:
X_train_minmax, X_test_minmax, y_train_minmax, y_test_minmax = train_test_split(test_minMaxdf_X, minMaxTest_Y,
                                                    stratify=Y,
                                                    random_state=3,
                                                    test_size=0.25,
                                                    shuffle=True)

In [86]:
X_train_minmax

id
25-1243311    URLs with dashes are not rendered correctly in...
25-2428876    Line change popup too small to read What steps...
25-381171     Starting an OSGi run configuration rebuilds al...
25-1437460    Terminal stop execution of process with shortc...
25-150873     Flex: If a Main.mxml is located in multiple im...
                                    ...                        
25-168009     Generate Persistence Mapping doesn't generate ...
25-1402011    Imports Added for Classes in Same Package The ...
25-653003     Unable to open .out file with IDEA 13.0.1 Step...
25-729866     Alternative "added line" visual The current po...
25-713119     sql: provide "qualify column" intention If col...
Length: 7902, dtype: object

In [94]:
from sklearn.ensemble import RandomForestRegressor
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.005, 0.01,.05, 0.06], max_df=[.5,.6,.7,.8], classifier=[ComplementNB(),LogisticRegression()], threshold=[.15,.05, 0.1])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [ ('tfid', TfidfVectorizer(min_df= params['min_df'], max_df= params['max_df'],
                                    tokenizer=LemmaTokenizer(),
                             )),
                             ('to_dense', DenseTransformer()),
                             ('clf',  params['classifier'])
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train_minmax, y_train_minmax)


    probabilitiesText1 = pipe.predict_proba(X_train_minmax)[:, 1]
    probabilitiesText2 = pipe.predict_proba(X_test_minmax)[:, 1]

    results.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=probabilitiesText1 > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=probabilitiesText2 > params['threshold'])
    ))


  0%|          | 0/96 [00:00<?, ?it/s]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/do

In [95]:
pd.DataFrame(results).sort_values('test_f1').style.bar(vmin=0, vmax=1)

,parameters,train_f1,test_f1
93,"{'classifier': LogisticRegression(), 'max_df': 0.8, 'min_df': 0.06, 'threshold': 0.15}",0.000000,0.000000
81,"{'classifier': LogisticRegression(), 'max_df': 0.7, 'min_df': 0.06, 'threshold': 0.15}",0.000000,0.000000
57,"{'classifier': LogisticRegression(), 'max_df': 0.5, 'min_df': 0.06, 'threshold': 0.15}",0.000000,0.000000
69,"{'classifier': LogisticRegression(), 'max_df': 0.6, 'min_df': 0.06, 'threshold': 0.15}",0.000000,0.000000
75,"{'classifier': LogisticRegression(), 'max_df': 0.7, 'min_df': 0.01, 'threshold': 0.15}",0.158508,0.014085
87,"{'classifier': LogisticRegression(), 'max_df': 0.8, 'min_df': 0.01, 'threshold': 0.15}",0.158508,0.014085
51,"{'classifier': LogisticRegression(), 'max_df': 0.5, 'min_df': 0.01, 'threshold': 0.15}",0.158508,0.014085
63,"{'classifier': LogisticRegression(), 'max_df': 0.6, 'min_df': 0.01, 'threshold': 0.15}",0.158508,0.014085
84,"{'classifier': LogisticRegression(), 'max_df': 0.8, 'min_df': 0.005, 'threshold': 0.15}",0.166667,0.014184
60,"{'classifier': LogisticRegression(), 'max_df': 0.6, 'min_df': 0.005, 'threshold': 0.15}",0.166667,0.014184


In [ ]:
pd.DataFrame(results).drop(columns='estimator').sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [71]:


#param_grid = dict(classifier=[ComplementNB(),LogisticRegression(),LinearRegression(), RandomForestRegressor()])




nb_pipeline = Pipeline(steps = [('scaler', MinMaxScaler()),
                                ('clf', ComplementNB())
                               ]
                       )

nb_pipeline.fit(X_train, y_train)

print (f'Train f1 = {f1_score(y_true=y_train, y_pred=nb_pipeline.predict(X_train))}')
print (f'Test f1 = {f1_score(y_true=y_test, y_pred=nb_pipeline.predict(X_test))}')

Train f1 = 0.039106145251396655
Test f1 = 0.0


In [74]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05])

results = []
for params in tqdm(ParameterGrid(param_grid)):
    probabilities1 = nb_pipeline.predict_proba(X_train)[:, 1]
    probabilities2 = nb_pipeline.predict_proba(X_test)[:, 1]
    results.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=probabilities1 > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=probabilities2 > params['threshold'])
    ))



100%|██████████| 8/8 [00:05<00:00,  1.48it/s]


In [76]:
pd.DataFrame(results).sort_values('test_f1').style.bar(vmin=0, vmax=1)

,parameters,train_f1,test_f1
2,{'threshold': 0.3},0.106667,0.000000
3,{'threshold': 0.4},0.055402,0.000000
4,{'threshold': 0.5},0.039106,0.000000
1,{'threshold': 0.2},0.214286,0.072727
6,{'threshold': 0.005},0.119097,0.099849
5,{'threshold': 0.01},0.150280,0.108075
7,{'threshold': 0.05},0.351906,0.147482
0,{'threshold': 0.1},0.355848,0.156134


In [ ]:


from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.01, .05], max_df=[.7, .8], scaler=[MinMaxScaler()])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [
        ('scaler', params['scaler']),
                             ('clf', ComplementNB())
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train, y_train)
    train_preds = pipe.predict(X_test)

    results.append(dict(
                        estimator=pipe,
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=pipe.predict(X_train)),
                        test_f1 = f1_score(y_true=y_test, y_pred=pipe.predict(X_test))
    ))


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.01, .05], max_df=[.7, .8])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [('tfidf', TfidfVectorizer(min_df=params['min_df'], max_df=params['max_df'],
                                                       token_pattern=r'[A-Za-z]{2,}',
                                                       stop_words=STOP_WORDS)),
                             ('to_dense', DenseTransformer()),
                             ('clf', ComplementNB())
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train, y_train)
    train_preds = pipe.predict(X_train)

    results.append(dict(
                        estimator=pipe,
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=pipe.predict(X_train)),
                        test_f1 = f1_score(y_true=y_test, y_pred=pipe.predict(X_test))
    ))

In [ ]:
pd.DataFrame(results).drop(columns='estimator').sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:
train_data1

In [ ]:
probabilities = pipe.predict_proba(X)[:, 1]

In [ ]:
probabilities

In [ ]:
threshold = 0.6

In [ ]:
f1_score(y_true=y, y_pred=probabilities > threshold)